In [0]:
!pip install spotipy

In [0]:
import pandas as pd
import time
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

# Credenciais fornecidas
client_id = '7e8c73e97d5b44cf9a10e4c1678a90b6'
client_secret = 'b1cc3f76eab24971b003d2a4fdec1e00'

# Autenticação com o Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

# Carregar o CSV
df = pd.read_csv('Musicas_com_Album_cleaned.csv')

# Função para buscar URL da capa com retry e backoff ajustados
def get_album_cover_url(artist, album):
    retry_delay = 5  # Tempo de espera inicial aumentado para 5 segundos
    max_retries = 10  # Número máximo de tentativas aumentado
    max_delay = 60 # Tempo máximo de espera entre as tentativas em segundos

    for attempt in range(max_retries):
        try:
            query = f"album:{album} artist:{artist}"
            results = sp.search(q=query, type='album', limit=1)
            items = results['albums']['items']
            if items:
                return items[0]['images'][0]['url']  # Imagem de maior resolução
            return None # Retorna None se não encontrar resultados
        except Exception as e:
            print(f"Erro ao buscar {album} de {artist} (Tentativa {attempt + 1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                # Calcula o próximo tempo de espera, limitando ao max_delay
                sleep_time = min(retry_delay * (2 ** attempt), max_delay)
                time.sleep(sleep_time)
            else:
                print(f"Falha ao buscar {album} de {artist} após {max_retries} tentativas.")
    return None

# Processar apenas a primeira linha de dados
if not df.empty:
    first_row = df.iloc[0]
    artist = first_row['Artist']
    album = first_row['Album']
    print(f"Processando a primeira linha de dados: {artist} - {album}")
    url = get_album_cover_url(artist, album)
    print(f"URL da capa encontrada: {url}")
else:
    print("O DataFrame está vazio.")

print("Teste concluído.")

In [0]:
import requests
import pandas as pd
import time

# Carregar o CSV (mantemos o carregamento do seu arquivo)
df = pd.read_csv('Musicas_com_Album_cleaned.csv')

# URL base da API da Deezer
DEEZER_API_BASE_URL = "https://api.deezer.com"

# Função para buscar URL da capa pela API da Deezer
def get_album_cover_url_deezer(artist, album):
    # Endpoint de busca por álbum
    search_endpoint = f"{DEEZER_API_BASE_URL}/search/album"
    params = {
        'q': f'artist:"{artist}" album:"{album}"'
    }

    try:
        response = requests.get(search_endpoint, params=params)
        response.raise_for_status()  # Lança exceção para status de erro (4xx ou 5xx)
        results = response.json()

        if results and results['data']:
            # Retorna a URL da capa do primeiro resultado encontrado (imagem extra grande)
            return results['data'][0]['cover_xl']
        return None # Retorna None se não encontrar resultados

    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição para a API da Deezer ao buscar {album} de {artist}: {e}")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado ao buscar {album} de {artist}: {e}")
        return None

# Aplicar a função em blocos com controle de tempo
urls = []
chunk_size = 500  # Definir o tamanho do bloco (pode ajustar conforme necessário)
# Iterar sobre o DataFrame em blocos
for i in range(0, len(df), chunk_size):
    chunk = df.iloc[i:i + chunk_size]
    print(f"Processando bloco de linhas {i} a {i + chunk_size - 1}...")
    for index, row in chunk.iterrows():
        artist = row['Artist']
        album = row['Album']
        url = get_album_cover_url_deezer(artist, album)
        urls.append(url)
        # Pequeno sleep entre requisições dentro do bloco para evitar picos
        time.sleep(0.1)
    # Tempo de espera maior entre os blocos para respeitar limites da API
    time.sleep(5) # Ajuste este tempo se encontrar problemas de limite de taxa

# Adicionar as URLs ao DataFrame
df['Album_Cover_URL'] = urls

# Salvar o DataFrame atualizado em um novo CSV
df.to_csv('Musicas_com_Capas_Deezer.csv', index=False)

print("Busca de capas pela API da Deezer concluída e arquivo salvo como 'Musicas_com_Capas_Deezer.csv'")

In [0]:
import pandas as pd

# Carregar o arquivo CSV
try:
    df_classic = pd.read_csv('ClassicHit.csv')
    print(f"Número original de linhas: {len(df_classic)}")

    # Remover linhas com valores faltantes ou falhos (NaN, None, etc.)
    df_classic_cleaned = df_classic.dropna()

    print(f"Número de linhas restantes após a limpeza: {len(df_classic_cleaned)}")
    print(f"Número de linhas eliminadas: {len(df_classic) - len(df_classic_cleaned)}")

    # Salvar o arquivo limpo em um novo CSV
    df_classic_cleaned.to_csv('ClassicHit_Cleaned.csv', index=False)

    print("Arquivo 'ClassicHit_Cleaned.csv' criado com sucesso.")

except FileNotFoundError:
    print("Erro: O arquivo 'ClassicHit.csv' não foi encontrado.")
except Exception as e:
    print(f"Ocorreu um erro durante o processamento: {e}")

In [0]:
import pandas as pd

# Carregar os dois arquivos CSV
try:
    df_classic_cleaned = pd.read_csv('ClassicHit_Cleaned.csv')
    df_deezer = pd.read_csv('Musicas_com_Capas_Deezer.csv')

    # Verificar as colunas em cada DataFrame
    print("Colunas em df_classic_cleaned:", df_classic_cleaned.columns)
    print("Colunas em df_deezer:", df_deezer.columns)

    # Realizar o merge dos DataFrames usando 'Artist' e 'Track'
    # Usaremos um merge left para manter todas as músicas de ClassicHit_Cleaned.csv
    merged_df = pd.merge(df_classic_cleaned, df_deezer[['Artist', 'Track', 'Album_Cover_URL']],
                         on=['Artist', 'Track'],
                         how='left')

    # Exibir as primeiras linhas do DataFrame mesclado para verificação
    print("\nPrimeiras linhas do DataFrame mesclado:")
    display(merged_df.head())

    # Salvar o DataFrame mesclado em um novo arquivo CSV
    merged_df.to_csv('ClassicHit_Cleaned_and_Added.csv', index=False)

    print("\nArquivo 'ClassicHit_Cleaned_and_Added.csv' criado com sucesso.")

except FileNotFoundError as e:
    print(f"Erro: Um dos arquivos não foi encontrado. {e}")
except KeyError as e:
    print(f"Erro: Coluna não encontrada ao tentar fazer o merge. Verifique se 'Artist', 'Track' e 'Album_Cover_URL' existem nos arquivos. {e}")
except Exception as e:
    print(f"Ocorreu um erro durante o processamento: {e}")